In [1]:
# Dependencies

import subprocess
import sys
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--user", package])

install("tensorflow")
subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "typing"])
install("setproctitle")
install("ray==1.7")
install("aioredis==1.3.1")
install("pandas")
install("modin")
install ("ray[serve]")
import numpy as np
from copy import copy
from scipy.stats import dirichlet
import json
import sys
import math
import ray
from ray import serve
import requests
import json
import time
from numpy.random import SeedSequence, default_rng

You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.


You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.


You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.


You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.


You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.


You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.


You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
# Constants

# My Mac has 8 CPUS
CPUS = 8
ray.init(num_cpus=CPUS, ignore_reinit_error=True)
serve.start()

# Constants for the game
ROWS = 6
COLS = 7
GAMENOTOVER = -2
PLAYER1 = 1
PLAYER2 = -1
PLAYERNONE = 0
MASK = np.array([3 ** i for i in range(42)])  # For efficiently hashing game states
HORIZONTAL = 0
VERTICAL = 1
FORWARD = 2
BACKWARD = 3

# Constants for data dimensions
STATE = 42
WINLOSS = 2
POLICY = 7
MAXSIZE = 60000  # Data is first in, first out

(pid=19895) 2021-11-01 23:54:47,092	INFO checkpoint_path.py:15 -- Using RayInternalKVStore for controller checkpoint and recovery.
(pid=19895) 2021-11-01 23:54:47,095	INFO http_state.py:75 -- Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:ILDTWR:SERVE_PROXY_ACTOR-node:10.18.205.43-0' on node 'node:10.18.205.43-0' listening on '127.0.0.1:8000'
2021-11-01 23:54:47,372	INFO api.py:455 -- Started Serve instance in namespace '010f0aea-d580-42b6-bfc2-dc47166464b3'.


In [3]:
# Hyperparameters

# Dirichlet hyperparameter, the dirichlet distribution adds noise to the probability vectors. If C > 1, vectors
# tend towards dense, even values
C = 2

# Noise does not inherently favor any particular column
ALPHA = np.full(COLS, 1)

# Weight of noise
EPSILON = 0.2

# Turn number at which the MCTS becomes deterministic
TAO = 10

# Number of MCTS simulations to choose a move
SIMULATIONS = 50

# Neural network hyperparameters
FILTERS = 75
L2PENALTY = 1e-4
MOMENTUM = 0.9  # Addition of momentum improves performance massively because the data is noisy in early stages
LR = 0.1
BATCHSIZE = 256

# Number of iterations of self play followed by retraining
ITERATIONS = 100

# Number of games to play before retraining.
EPISODES = 50

In [4]:
# Hashable wrapper class for a np array representing a board
class HashableArray:
  def __init__(self, arr):
    self._arr = arr

  def get_arr(self):
    return self._arr

  def __hash__(self):
    return int(sum(MASK * (self._arr + 1)))

  def __eq__(self, other):
    return np.all(self._arr == other._arr)

In [5]:
# Manages game state.
class ConnectFour: 
  def __init__(self):
    self._board = np.zeros((ROWS, COLS))
    self._flip = np.zeros((ROWS, COLS))
    self._reverse_board = np.zeros((ROWS, COLS))
    self._reverse_flip = np.zeros((ROWS, COLS))

    self._height = np.zeros(COLS, dtype=np.int8)
    self._next = 1
    self._undo_list = np.zeros((ROWS * COLS, 2))
    self._turn = 0
    self._winner = -2

  # Makes a move in the column.
  def move(self, col):
    if self._height[col] >= ROWS:
      # col is a value above 6
      self.move((col + 1) % COLS)
      return False
    else:
      self._board[int(self._height[col])][col] = self._next
      self._flip[int(self._height[col])][COLS - col - 1] = self._next

      self._reverse_board[int(self._height[col])][col] = self._next * -1
      self._reverse_flip[int(self._height[col])][COLS - col - 1] = self._next * -1
      self._undo_list[self._turn] = np.array([self._height[col], col])

      checks = [self._check4(int(self._height[col]), col, dir) for dir in range(4)]
      self._turn += 1
      self._height[col] += 1
      self._next *= -1
      if np.sum(checks) > 0:
        self._winner = PLAYER1
      elif np.sum(checks) < 0:
        self._winner = PLAYER2
      elif np.sum(self._height) == ROWS * COLS:
        self._winner = PLAYERNONE
      return True

  # Restores the game state before the last move.
  def undo(self):
    row, col = tuple(self._undo_list[self._turn - 1])
    row = int(row)
    col = int(col)
    self._board[row][col] = PLAYERNONE
    self._flip[row][COLS - col - 1] = PLAYERNONE
    self._reverse_board[row][col] = PLAYERNONE
    self._reverse_flip[row][COLS - col - 1] = PLAYERNONE
    
    self._undo_list[self._turn - 1] = np.zeros(2)
    self._turn -= 1
    self._height[col] -= 1
    self._next *= -1
    self._winner = GAMENOTOVER

  # Returns PLAYER1 if player 1 is the winner in the current game state, PLAYER2 if
  # player 2 is the winner in the current game state, PLAYERNONE if the game is a draw, GAMENOTOVER if the game is not
  # decided yet.
  def get_winner(self):
    return self._winner

  # Returns the current turn number, starting with 0
  def get_turn(self):
    return self._turn

  # Returns the board as a HashableArray. The board returned appears to the next player as if they
  # are player 1. So if the next player is player 2, the pieces are flipped. This is done so the neural network
  # can always understand itself as player 1.
  def get_board(self):
    if (self._next == PLAYER1):
      return HashableArray(self._board.flatten())
    else:
      return HashableArray(self._reverse_board.flatten())

  # Returns the board reflected horizontally. Connect Four is the same game reflected horizontally, so the neural
  # network benefits from augmenting the data by flipping all boards.
  def get_flip(self):
    if (self._next == PLAYER1):
      return HashableArray(self._flip.flatten())
    else:
      return HashableArray(self._reverse_flip.flatten())

  # Private helper function checks a particular game location for a 4 in a row. Checks both directions around the
  # input row and col in the orientation given by the input direction.
  def _check4(self, row, col, direction):
    if direction == HORIZONTAL:
      dr = 0
      dc = 1
    elif direction == VERTICAL:
      dr = 1
      dc = 0
    elif direction == FORWARD:
      dr = 1
      dc = -1
    else: # direction = BACKWARD
      dr = 1
      dc = 1
    player_at = self._board[row][col]
    length = 1
    i = 1
    j = -1
    while 0 <= row + i * dr < ROWS and 0 <= col + i * dc < COLS \
          and self._board[row + i * dr][col + i * dc] == player_at:
      i += 1
      length += 1
      if length == 4:
        return player_at
    while 0 <= row + j * dr < ROWS and 0 <= col + j * dc < COLS \
          and self._board[row + j * dr][col + j * dc] == player_at:
      j -= 1
      length += 1
      if length == 4:
        return player_at
    return PLAYERNONE

(pid=19897) INFO:     Started server process [19897]


In [6]:
# Manages the data and trains the neural network.
class Trainer:
    def __init__(self, file=None):
      self._data = np.array([])
      if file != None:
        new_data = json.load(open(file, 'r'))
        self.retrain(new_data)

    def retrain(self, new_data):
      for game_data in new_data:
        self._data = np.append(self._data, game_data)
      self._data = self._data.reshape((-1, STATE + WINLOSS + POLICY))
      print(len(self._data))
      if len(self._data) > MAXSIZE:
        extra_size = len(self._data) - MAXSIZE
        self._data = self._data[extra_size:, :]
      import tensorflow as tf
      from tensorflow import keras
      X = self._data[:,:STATE]
      y1 = self._data[:,STATE:STATE + WINLOSS]
      y2 = self._data[:,STATE + WINLOSS:]
      print('retraining')
      loaded = tf.keras.models.load_model('initial')       
      loaded.fit(X, (y1, y2), batch_size=BATCHSIZE)
      loaded.save('trained')
      
    def save(self, iteration):
      json.dump(self._data.tolist(), open('data' + str(iteration) + '.txt', 'w'))

In [7]:
# Defines the neural network architecture.
def initialize_network():
  # Ray has problems with tensorflow if tensorflow is imported globally
  import tensorflow as tf
  from tensorflow import keras
  from keras import regularizers
  from keras import optimizers

  # The first layers of the network
  def conv_layer(input_layer):
    reshape = keras.layers.Reshape((ROWS, COLS, 1), input_shape=(ROWS * COLS,))(input_layer)
    conv = keras.layers.Conv2D(filters=FILTERS, kernel_size=4, strides=1,
                               padding='same', input_shape=(ROWS, COLS, 1),
                               kernel_regularizer=regularizers.l2(L2PENALTY))(reshape)
    norm = tf.keras.layers.BatchNormalization()(conv)
    rect = tf.keras.layers.PReLU()(norm)
    return rect

  # A repeated structure of layers with the same input and output shape
  def res_layer(input_layer):
    conv1 = keras.layers.Conv2D(filters=FILTERS, kernel_size=4, strides=1,
                                padding='same', input_shape=(ROWS, COLS, FILTERS),
                                kernel_regularizer=regularizers.l2(L2PENALTY))(input_layer)
    norm1 = tf.keras.layers.BatchNormalization()(conv1)
    rect1 = tf.keras.layers.PReLU()(norm1)
    conv2 = keras.layers.Conv2D(filters=FILTERS, kernel_size=4, strides=1,
                                padding='same', input_shape=(ROWS, COLS, FILTERS),
                                kernel_regularizer=regularizers.l2(L2PENALTY))(rect1)
    norm2 = tf.keras.layers.BatchNormalization()(conv2)
    concat = keras.layers.concatenate([input_layer, norm2])
    rect2 = tf.keras.layers.PReLU()(concat)
    return rect2

  # Outputs the q value
  def value_head(input_layer):
    conv = keras.layers.Conv2D(filters=1, kernel_size=1, strides=1,
                               padding='same', input_shape=(ROWS, COLS, FILTERS),
                               kernel_regularizer=regularizers.l2(L2PENALTY))(input_layer)
    norm = tf.keras.layers.BatchNormalization()(conv)
    rect = tf.keras.layers.PReLU()(norm)
    flat = keras.layers.Flatten()(rect)
    dense = keras.layers.Dense(20, kernel_regularizer=regularizers.l2(L2PENALTY))(flat)
    rect2 = tf.keras.layers.PReLU()(dense)
    value = keras.layers.Dense(1, kernel_regularizer=regularizers.l2(L2PENALTY),
                               activation='tanh')(rect2)
    return value

  # Outputs the probability vector
  def policy_head(input_layer):
    conv = keras.layers.Conv2D(filters=2, kernel_size=1, strides=1,
                               padding='same', input_shape=(ROWS, COLS, FILTERS),
                               kernel_regularizer=regularizers.l2(L2PENALTY))(input_layer)
    norm = tf.keras.layers.BatchNormalization()(conv)
    rect = tf.keras.layers.PReLU()(norm)
    flat = keras.layers.Flatten()(rect)
    policy = keras.layers.Dense(COLS, kernel_regularizer=regularizers.l2(1e-4), activation='softmax')(flat)
    return policy

  input_layer = keras.layers.Input(shape=(ROWS * COLS,))
  conv = conv_layer(input_layer)
  res1 = res_layer(conv)
  res2 = res_layer(res1)
  res3 = res_layer(res2)
  res4 = res_layer(res3)
  res5 = res_layer(res4)
  res6 = res_layer(res5)
  value = value_head(res6)
  policy = policy_head(res6)
  network = keras.Model(inputs=[input_layer], outputs=[value, policy])
  optimizer = tf.keras.optimizers.SGD(learning_rate=LR, momentum=MOMENTUM)
  network.compile(loss=['mean_squared_error', 'categorical_crossentropy'],
                  optimizer=optimizer)
  network.save('initial')
  network.save('trained')

In [8]:
# ADT representing statistics about a game state that MCTS uses to evaluate moves to explore.
class Node:
  def __init__(self):
    self.q = 0
    self.probabilities = np.zeros(COLS)
    self.n = 0
    self.children = []

In [9]:
# MCTS

# Calculate the initial statistics for a node.
def first_visit(g, node, nodes, network):
    output = network.predict(g.get_board().get_arr().reshape((1, ROWS * COLS)))
    node.q = output[0][0][0]
    node.probabilities = np.array(output[1][0], dtype=float)
    
    # Add each child
    for move in range(COLS):
      if g.move(move):
        move_board = g.get_board()
        # If this game state has been found by a different order, jump in the tree to that node
        if move_board in nodes.keys():
          child = nodes[move_board]
        else:
          child = Node()
          nodes[move_board] = child
      else:
        # The case that a column is full
        child = None
      node.children.append(child)
      g.undo()
    node.n = 1

# Updates the quality of a node and increment its visit count
def update(node, v):
  node.q = (node.q * node.n + v) / (node.n + 1);
  node.n += 1

# Returns an array of the number of visits to each child node
def get_visits(node):
  visits = []
  for child in node.children:
    if not child is None:
      visits.append(child.n)
    else:
      visits.append(0)
  return np.array(visits)

# Chooses what move is most worthy of exploration.
def choose_move_search(node):
  bounds = np.zeros(COLS)
  for i in range(COLS):
    child = node.children[i]
    if not child is None:
      # Get statistics
      q_action = child.q
      n_action = child.n
      p_action = node.probabilities[i]

      # PUCT: Balances exploration vs exploitation
      score = q_action + C * p_action * node.n ** 1/2 / (n_action + 1)
      bounds[i] = score
    else:
      bounds[i] = -sys.float_info.max
  return np.argmax(bounds), node.children[np.argmax(bounds)]

# Performs one MCTS simulation
def search(g, node, nodes, network):
  result = g.get_winner()
  if result != GAMENOTOVER:
    # Game end state, backpropegate the result
    node.q += 1
    node.n += 1
    if result == PLAYERNONE:
      return 0
    else:
      return -1
  elif node.n == 0:
    # New game state, backpropegate the result
    first_visit(g, node, nodes, network)
    return -node.q
  else:
    # Seen this game state before, pick the best move to explore and continue building the tree
    move, next_node = choose_move_search(node)
    g.move(move)
    v = search(g, next_node, nodes, network)
    g.undo()
    update(node, v)
    return -v

# Adds dirichlet noise to the probability vector
def noise(root, rng):
  p_vector = root.probabilities
  noise_vector = dirichlet.rvs(ALPHA, size=1, random_state=rng)[0]
  root.probabilities = ((1 - EPSILON) * p_vector) + ((EPSILON) * noise_vector)
  return root

# Evaluates the move to play based on the search results. Before turn TAO, samples from the visits to each child
# node as a probability vector. After turn TAO, chooses the most visited move
def final_move(root, rng, sample):
  visits = get_visits(root)
  max_val = np.max(visits)
  indices = [index for index, value in enumerate(visits) if value == max_val]
  if len(indices) == 1:
    if sample:
      return rng.choice(COLS, p=visits / sum(visits)), visits
    else:
      return indices[0], visits
  else:
    return -1, visits

# Performs a MCTS to choose a move
def get_move(g, rng, network):
  root = Node()
  nodes = dict()
  first_visit(g, root, nodes, network)
  stochasticity = g.get_turn() < TAO
  if stochasticity:
    root = noise(root, rng)
  for i in range(SIMULATIONS):
    search(g, root, nodes, network)
  move, visits = final_move(root, rng, stochasticity)
  while move == -1:
    search(g, root, nodes, network)
    move, visits = final_move(root, rng, stochasticity)
  return move, visits / np.sum(visits)

In [10]:
# Records a game state into the data
def record(new_data, g, pi, flip_pi, result):
  board = g.get_board().get_arr()
  flip = g.get_flip().get_arr()
  if result == 1:
    labels = np.array([1, 0])
  elif result == -1:
    labels = np.array([0, 1])
  else:
    labels = np.array([0.5, 0.5])
  norm_labels = np.append(labels, pi)
  flip_labels = np.append(labels, flip_pi)
  new_data = np.append(new_data, np.append(board, norm_labels))
  new_data = np.append(new_data, np.append(flip, flip_labels))
  return new_data

# Plays a game
def play_game(g, rng, network):
  winner = g.get_winner()
  if winner != GAMENOTOVER:
    if winner == PLAYERNONE:
      return 0, np.array([])
    else:
      return -1, np.array([])
  else:
    move, pi = get_move(g, rng, network)
    print(str(move), pi)
    flip_pi = np.flip(pi)
    g.move(move)
    result, new_data = play_game(g, rng, network)
    g.undo()
    new_data = record(new_data, g, pi, flip_pi, result)
    return -result, new_data

# Executes an episode of self play
@ray.remote
def self_play(seed):
  import tensorflow as tf
  from tensorflow import keras
  rng = default_rng(seed)
  print('playing')
  g = ConnectFour()
  network = tf.keras.models.load_model('trained')
  result, new_data = play_game(g, rng, network)
  return new_data

In [11]:
# Runs the training process
initialize_network()
trainer = Trainer()
ss = SeedSequence(12345)
seeds = ss.spawn(ITERATIONS * CPUS)
for iteration in range(ITERATIONS):
  futures = []
  for i in range(EPISODES):
    # Limit the number of concurrent processes to the number of CPUS
    if i > CPUS:
      num_ready = i - CPUS
      ray.wait(futures, num_returns=num_ready)
    futures.append(self_play.remote(seeds[iteration * CPUS + i]))
  new_data = ray.get(futures)
  trainer.retrain(new_data)
  trainer.save(iteration)

2021-11-01 23:54:51.412991: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-01 23:54:54.909292: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: initial/assets
INFO:tensorflow:Assets written to: trained/assets
(self_play pid=19896) playing
(self_play pid=19891) playing
(self_play pid=19892) playing
(self_play pid=19890) playing
(self_play pid=19894) playing
(self_play pid=19893) playing


(pid=19890) 2021-11-01 23:55:15.053889: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(pid=19890) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=19892) 2021-11-01 23:55:15.027264: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(pid=19892) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=19891) 2021-11-01 23:55:15.048200: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(pid=1

(self_play pid=19908) playing
(self_play pid=19909) playing


(pid=19908) 2021-11-01 23:55:16.018157: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(pid=19908) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=19909) 2021-11-01 23:55:16.094316: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(pid=19909) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=19890) 2021-11-01 23:55:23.396220: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
(pid=19892) 2021-11-01 23:55:23.396698: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:1

...

(self_play pid=20439) 6 [0.16 0.16 0.12 0.12 0.12 0.12 0.2 ]
(self_play pid=20039) 2 [0.12 0.12 0.12 0.16 0.12 0.2  0.16]
(self_play pid=20038) 6 [0.14 0.14 0.18 0.16 0.14 0.12 0.12]
(self_play pid=20053) 5 [0.15384615 0.11538462 0.13461538 0.11538462 0.17307692 0.15384615
(self_play pid=20053)  0.15384615]
(self_play pid=20437) 3 [0.12 0.16 0.18 0.16 0.1  0.12 0.16]
(self_play pid=20439) 0 [0.19230769 0.11538462 0.17307692 0.15384615 0.09615385 0.15384615
(self_play pid=20439)  0.11538462]
(self_play pid=19890) 4 [0.16 0.18 0.14 0.12 0.14 0.12 0.14]
(self_play pid=20039) 5 [0.12 0.16 0.12 0.14 0.12 0.2  0.14]
(self_play pid=20038) 3 [0.12 0.16 0.12 0.12 0.2  0.12 0.16]
(self_play pid=22136) 0 [0.14 0.16 0.18 0.16 0.12 0.12 0.12]
(self_play pid=22138) 5 [0.14 0.16 0.12 0.16 0.12 0.12 0.18]
(self_play pid=20053) 1 [0.19298246 0.12280702 0.14035088 0.1754386  0.10526316 0.14035088
(self_play pid=20053)  0.12280702]
(self_play pid=20437) 2 [0.12 0.2  0.16 0.12 0.12 0.14 0.14]
(self_play p

(self_play pid=20437) 6 [0.19607843 0.11764706 0.11764706 0.1372549  0.1372549  0.17647059
(self_play pid=20437)  0.11764706]
(self_play pid=20439) 4 [0.16363636 0.16363636 0.16363636 0.16363636 0.18181818 0.16363636
(self_play pid=20439)  0.        ]
(self_play pid=20038) 1 [0.12 0.2  0.16 0.12 0.1  0.16 0.14]
(self_play pid=19890) 4 [0.16363636 0.16363636 0.16363636 0.         0.18181818 0.16363636
(self_play pid=19890)  0.16363636]
(self_play pid=22136) 6 [0.11111111 0.11111111 0.12962963 0.16666667 0.18518519 0.14814815
(self_play pid=22136)  0.14814815]
(self_play pid=20039) 4 [0.18 0.2  0.12 0.14 0.12 0.12 0.12]
(self_play pid=22138) 6 [0.12280702 0.15789474 0.12280702 0.15789474 0.1754386  0.14035088
(self_play pid=22138)  0.12280702]
(self_play pid=20437) 0 [0.16 0.16 0.12 0.18 0.12 0.12 0.14]
(self_play pid=20439) 3 [0.16363636 0.16363636 0.16363636 0.18181818 0.16363636 0.16363636
(self_play pid=20439)  0.        ]
(self_play pid=20038) 6 [0.13461538 0.11538462 0.19230769 0.1

(self_play pid=22136) 3 [0.15789474 0.14035088 0.14035088 0.1754386  0.12280702 0.15789474
(self_play pid=22136)  0.10526316]
(self_play pid=20439) 3 [0.04 0.02 0.   0.86 0.04 0.04 0.  ]
(self_play pid=20439) playing
(self_play pid=20053) 3 [0.12 0.14 0.12 0.12 0.14 0.2  0.16]
(self_play pid=20437) 4 [0.14 0.14 0.14 0.14 0.16 0.14 0.14]
(self_play pid=20039) 4 [0.14 0.14 0.14 0.14 0.16 0.14 0.14]
(self_play pid=20038) 5 [0.14035088 0.14035088 0.14035088 0.14035088 0.14035088 0.15789474
(self_play pid=20038)  0.14035088]
(self_play pid=22138) 2 [0.12 0.12 0.16 0.16 0.12 0.14 0.18]
(self_play pid=19890) 2 [0.16 0.14 0.18 0.12 0.14 0.12 0.14]
(self_play pid=22136) 4 [0.14 0.14 0.14 0.14 0.16 0.14 0.14]
(self_play pid=20038) 4 [0.14 0.14 0.12 0.14 0.22 0.14 0.1 ]
(self_play pid=20053) 4 [0.14 0.12 0.14 0.14 0.18 0.16 0.12]
(self_play pid=20437) 6 [0.14285714 0.14285714 0.14285714 0.14285714 0.125      0.14285714
(self_play pid=20437)  0.16071429]
(self_play pid=20039) 0 [0.18181818 0.16363

KeyboardInterrupt: 

(self_play pid=20038) 0 [0.15384615 0.17307692 0.11538462 0.19230769 0.11538462 0.11538462
(self_play pid=20038)  0.13461538]
